# Assignment2, CMPT826

* Seyedeh Mina Mousavifar
* 11279515
* sem311

##  STEP 1: Preparation

### Data Collection

In [2]:
import pandas as pd
import sqlalchemy as db

engine = db.create_engine('mysql://sem311:p^HA89/h@crepe.usask.ca:3306/SHED10')

connection = engine.connect()
metadata = db.MetaData()

# Equivalent to 'SELECT * FROM battery
battery = db.Table('battery', metadata, autoload=True, autoload_with=engine)
query = db.select([battery])

# getting data by executing the query above
BatteryResultProxy = connection.execute(query)
BatteryResultSet = BatteryResultProxy.fetchall()

# converting data to data frame
battery_data = pd.DataFrame(BatteryResultSet)
battery_data.columns = BatteryResultSet[0].keys()

# removing index column from data - 0 for rows and 1 for column
battery_data = battery_data.drop('index', 1)

# Equivalent to 'SELECT * FROM gps
gps = db.Table('gps', metadata, autoload=True, autoload_with=engine)
query = db.select([gps])

# getting data by executing the query above
GPSResultProxy = connection.execute(query)
GPSResultSet = GPSResultProxy.fetchall()

# converting data to data frame
gps_data = pd.DataFrame(GPSResultSet)
gps_data.columns = GPSResultSet[0].keys()

# removing index column from data - 0 for rows and 1 for column
gps_data = gps_data.drop('index', 1)

print('fetching data completed')

fetching data completed


### Filtering Data

In [3]:
# counting number of battery information per user
battery_info = battery_data.groupby(['user_id']).size().reset_index(name='record_count')

# calculating filtering cutoff
cutoff_percentage = 0.5
max_battery_info = (60 / 5) * 24 * 30
battery_cutoff = cutoff_percentage * max_battery_info

# filtering users with less than 50%
battery_info_50 = battery_info.loc[battery_info['record_count'] > battery_cutoff]
users_filter_50, _ = battery_info_50.shape

# preserving only users with more than 50% battery record
user_battery = pd.merge(left=battery_data, right=battery_info_50, 
                           left_on='user_id', right_on='user_id')

# filtering accuracy more than 100
gps_data = gps_data.loc[gps_data['accu'] < 100]

# outside latitude range
gps_data = gps_data.loc[gps_data['lat'] > 52.058366]
gps_data = gps_data.loc[gps_data['lat'] < 52.214609]

# outside longitude range
gps_data = gps_data[gps_data['lon'] > -106.7649138128]
gps_data = gps_data.loc[gps_data['lon'] < -106.52225319]

# creating dataframe for filtering Saskatoon data for preferred users
good_50_user_id = user_battery.user_id.unique()
gps_data = gps_data[gps_data.user_id.isin(good_50_user_id)]

### Stratify and Aggregate

In [4]:
import datetime
import math
from pyproj import Proj
import numpy as np

# removing unnecessary columns
gps_data = gps_data.drop(['accu', 'alt', 
                          'bearing', 'speed', 
                          'record_time_minute', 
                          'timestamp', 'pokemon'], 1)

# calculating mean of latitude and longitude for every duty cycle
gps_data = gps_data.astype({'lat': 'float64', 'lon': 'float64'})

# sorting based on time 
gps_data = gps_data.sort_values(['user_id', 'record_time'])

# removing December test data
gps_data['Date'] = pd.to_datetime(gps_data['record_time']).dt.date
testdate = datetime.datetime.strptime('2016-12-09', "%Y-%m-%d").date()
gps_data = gps_data[(gps_data['Date'] > testdate)].dropna().reset_index()
gps_data = gps_data.drop(['Date', 'index'],1)


# extract duty cycles and calculate mean of latitude and longitude
gps_data = gps_data.groupby('user_id').resample('5T', on='record_time').mean()
gps_data = gps_data.drop(['user_id'],1).reset_index()

# converting to UTM
myproj = Proj('epsg:32613', proj='utm', zone=13, 
              ellps='WGS84', preserve_units=True)

gps_data['x'], gps_data['y'] = myproj(gps_data['lon'].values, 
                                      gps_data['lat'].values)
# binning
GRID_SIZE = 100

# find grid start point
start_x, start_y = gps_data.x.min(), gps_data.y.min()

# labeling grids
gps_data['x_grid'] = np.ceil((gps_data['x'] - start_x)/GRID_SIZE)
gps_data['y_grid'] = np.ceil((gps_data['y'] - start_y)/GRID_SIZE)

# sort data
gps_data = gps_data.sort_values(['user_id', 'record_time']).dropna()

# creating grid cell labels (x,y)
gps_data = gps_data.astype({'x_grid': 'int32', 
                            'y_grid': 'int32'}).astype({'x_grid': 'str', 
                                                        'y_grid': 'str'})
gps_data['grid_label'] = gps_data['x_grid'] + ',' + gps_data['y_grid']
gps_data = gps_data.astype({'x_grid': 'int32', 'y_grid': 'int32'})

##  STEP 2: Trip Definition

### Operationalizing trips

For finding trips, first, I compare two consecutive rows for a user and discover whether it had changed its cell. Then, I gave id for these change cells to make them traceable as different trajectories. So, the stayed time in each cell can be calculated by grouping based on a specific user, grid and trajectory id. Afterwards, trips can be operationalized as staying in cells less than a certain threshold.

In [ ]:
gps_data = pd.read_pickle('data/gps_info.pkl')

In [122]:
# change cell condition
gps_data['change_cell'] = np.where(gps_data['grid_label'] != gps_data['grid_label'].shift(1), 1, 0)
gps_data['change_cell'] = np.where(gps_data['user_id'] == gps_data['user_id'].shift(1), 
                                   gps_data['change_cell'], 0)

# change in a cell can be used as change id for showing user trajectory
gps_data['change_id'] = gps_data.groupby('user_id')['change_cell'].cumsum()

#### $N = 1$
This means that staying in a cell more than 1 duty cycle ends the trip and leaving the cell less than or equal to 1 duty cycle start a trip.

In [123]:
TRIP_N = 1

# calculating staying time in one cell during their change
gps_trip_1 = gps_data.groupby(['user_id', 
                            'grid_label', 
                            'change_id']).size().reset_index(name = 'stay_count')

gps_trip_1['is_dwell'] = np.where(gps_trip_1['stay_count'] > TRIP_N, 1, 0)

# sorting data based on time
gps_trip_1 = gps_trip_1.sort_values(['user_id', 'change_id'])
gps_trip_1 = gps_trip_1.reset_index().drop(['index'], 1)

# trips are calculated as time between dwells
gps_trip_1['trip_id'] = gps_trip_1.groupby('user_id')['is_dwell'].cumsum()

gps_trip_1.head(10)

,user_id,grid_label,change_id,stay_count,is_dwell,trip_id
0,514,"59,69",0,1,0,0
1,514,"59,70",1,1,0,0
2,514,"59,69",2,3,1,1
3,514,"59,70",3,1,0,1
4,514,"59,69",4,5,1,2
5,514,"59,70",5,4,1,3
6,514,"59,69",6,1,0,3
7,514,"59,70",7,1,0,3
8,514,"59,69",8,2,1,4
9,514,"59,70",9,1,0,4


#### Trip length
This is the number of different grids seen until dwelling.

In [124]:
# trip length
trip_info_1 = gps_trip_1.groupby(['user_id',
                              'trip_id']).size().reset_index(name = 'length')
trip_info_1.head(10)

,user_id,trip_id,length
0,514,0,2
1,514,1,2
2,514,2,1
3,514,3,3
4,514,4,4
5,514,5,2
6,514,6,2
7,514,7,2
8,514,8,1
9,514,9,3


#### Number of Trips

In [125]:
trip_num_1 = trip_info_1.groupby('user_id').size().reset_index(name = 'count')
trip_num_1.head(10)

,user_id,count
0,514,814
1,534,197
2,551,717
3,555,409
4,559,307
5,767,130
6,793,228
7,933,315
8,942,196
9,943,208


#### Time of each trip

In this situation that $N=1$ the length of the trip is its time.

In [126]:
# Trip time
trip_info_1['trip_time'] = trip_info_1['length']

trip_info_1.head(10)

,user_id,trip_id,length,trip_time
0,514,0,2,2
1,514,1,2,2
2,514,2,1,1
3,514,3,3,3
4,514,4,4,4
5,514,5,2,2
6,514,6,2,2
7,514,7,2,2
8,514,8,1,1
9,514,9,3,3


#### $N = 3$
This means that staying in a cell more than 3 duty cycle ends the trip and leaving the cell less than or equal to 3 duty cycle start a trip.

In [127]:
TRIP_N = 3

# calculating staying time in one cell during their change
gps_trip_3 = gps_data.groupby(['user_id', 
                            'grid_label', 
                            'change_id']).size().reset_index(name = 'stay_count')

gps_trip_3['is_dwell'] = np.where(gps_trip_3['stay_count'] > TRIP_N, 1, 0)

# sorting data based on time
gps_trip_3 = gps_trip_3.sort_values(['user_id', 'change_id'])
gps_trip_3 = gps_trip_3.reset_index().drop(['index'], 1)

# trips are calculated as time between dwells
gps_trip_3['trip_id'] = gps_trip_3.groupby('user_id')['is_dwell'].cumsum()

#### Trip length

In [142]:
# trip length
trip_info_3 = gps_trip_3.groupby(['user_id',
                              'trip_id']).size().reset_index(name = 'length')
trip_info_3.head(10)

,user_id,trip_id,length
0,514,0,4
1,514,1,1
2,514,2,11
3,514,3,2
4,514,4,12
5,514,5,2
6,514,6,2
7,514,7,2
8,514,8,5
9,514,9,9


#### Number of Trips

In [130]:
trip_num_3 = trip_info_3.groupby('user_id').size().reset_index(name = 'count')
trip_num_3.head(10)

,user_id,count
0,514,506
1,534,140
2,551,467
3,555,249
4,559,203
5,767,87
6,793,139
7,933,198
8,942,120
9,943,123


#### Time of each trip

In this situation that $N=3$, the sum of staying in each cell involved in the trip is the trip time.

In [143]:
# Trip time

# change dwellings that are more than threshold to 1 to 
# calculate them as just 1 duty cycle just as entering the cell
tmp_trip = gps_trip_3.copy(deep=True)
tmp_trip.loc[tmp_trip.stay_count > TRIP_N, 'stay_count'] = 1

trip_time_3 = tmp_trip.groupby(['user_id', 'trip_id']).agg(time = ('stay_count', 'sum'))
trip_time_3 = trip_time_3.reset_index()

# add it trip length to have full trip info
trip_info_3 = pd.merge(trip_info_3, trip_time_3, how='inner', on=['user_id', 'trip_id'])
trip_info_3.head(10)

,user_id,trip_id,length,time
0,514,0,4,6
1,514,1,1,1
2,514,2,11,15
3,514,3,2,2
4,514,4,12,17
5,514,5,2,2
6,514,6,2,3
7,514,7,2,2
8,514,8,5,6
9,514,9,9,13


#### $N = 5$
This means that staying in a cell more than 3 duty cycle ends the trip and leaving the cell less than or equal to 3 duty cycle start a trip.

In [148]:
TRIP_N = 5

# calculating staying time in one cell during their change
gps_trip_5 = gps_data.groupby(['user_id', 
                            'grid_label', 
                            'change_id']).size().reset_index(name = 'stay_count')

gps_trip_5['is_dwell'] = np.where(gps_trip_5['stay_count'] > TRIP_N, 1, 0)

# sorting data based on time
gps_trip_5 = gps_trip_5.sort_values(['user_id', 'change_id'])
gps_trip_5 = gps_trip_5.reset_index().drop(['index'], 1)

# trips are calculated as time between dwells
gps_trip_5['trip_id'] = gps_trip_5.groupby('user_id')['is_dwell'].cumsum()

#### Trip length

In [149]:
# trip length
trip_info_5 = gps_trip_5.groupby(['user_id',
                              'trip_id']).size().reset_index(name = 'length')
trip_info_5.head(10)

,user_id,trip_id,length
0,514,0,18
1,514,1,12
2,514,2,4
3,514,3,2
4,514,4,5
5,514,5,9
6,514,6,6
7,514,7,6
8,514,8,2
9,514,9,2


#### Number of Trips

In [150]:
trip_num_5 = trip_info_5.groupby('user_id').size().reset_index(name = 'count')
trip_num_5.head(10)

,user_id,count
0,514,333
1,534,123
2,551,353
3,555,180
4,559,165
5,767,83
6,793,113
7,933,151
8,942,90
9,943,92


#### Time of each trip

In this situation that $N=5$, the sum of staying in each cell involved in the trip is the trip time.

In [151]:
# Trip time

# change dwellings that are more than threshold to 1 to 
# calculate them as just 1 duty cycle just as entering the cell
tmp_trip = gps_trip_5.copy(deep=True)
tmp_trip.loc[tmp_trip.stay_count > TRIP_N, 'stay_count'] = 1

trip_time_5 = tmp_trip.groupby(['user_id', 'trip_id']).agg(time = ('stay_count', 'sum'))
trip_time_5 = trip_time_5.reset_index()

# add it trip length to have full trip info
trip_info_5 = pd.merge(trip_info_5, trip_time_5, how='inner', on=['user_id', 'trip_id'])
trip_info_5.head(10)

,user_id,trip_id,length,time
0,514,0,18,35
1,514,1,12,17
2,514,2,4,8
3,514,3,2,2
4,514,4,5,6
5,514,5,9,13
6,514,6,6,9
7,514,7,6,16
8,514,8,2,2
9,514,9,2,5
